# Hybrid Classical Quantum AutoEncoder for anomaly detection

## VQC
The VQC implemented is the number 10 of the reference paper, which essentially is a stack Pauli $YX$ rotation gate and a circular series of controlled $CX$, stacked while alternated with an encoding based on the Pauli $X$ rotation

In [ ]:
from qiskit.circuit import QuantumCircuit, Gate, ParameterVector
from qiskit.opflow.expectations import PauliExpectation
from qiskit_machine_learning.connectors import TorchConnector

def get_encoding_block(nqubits: int, features: ParameterVector) -> Gate:
    """ n parameters required """
    assert len(features) == nqubits 
    block = QuantumCircuit(nqubits, name="Encoding Block")
    for i in range(nqubits):
        block.rx(features[i], i)
    return block.to_gate()

def get_ansatz_block(nqubits: int, parameters: ParameterVector) -> Gate:
    """
    need 2n parameters
    """
    assert nqubits * 2 == len(parameters)
    block = QuantumCircuit(nqubits, name="Ansatz Block")
    for i in range(nqubits):
        block.ry(parameters[i], i)
        block.rx(parameters[i + nqubits], i)
    if nqubits > 1:
        block.cx(nqubits - 1, 0)
        for i in range(nqubits - 1):
            block.cx(i, i + 1)
    return block.to_gate()

def get_ansatz(nqubits: int, parameters: ParameterVector, features: ParameterVector, reps: int=3) -> QuantumCircuit:
    assert len(parameters) == reps * 2 * nqubits
    ansatz = QuantumCircuit(nqubits)
    ansatz.compose(get_ansatz_block(nqubits, parameters[:2 * nqubits]), range(nqubits), inplace=True)
    for i in range(1, reps):
        ansatz.barrier()
        ansatz.compose(get_encoding_block(nqubits, features), range(nqubits), inplace=True)
        ansatz.barrier()
        ansatz.compose(get_ansatz_block(nqubits, parameters[2 * nqubits * i :2 * nqubits * (i + 1)]), range(nqubits), inplace=True)
    return ansatz

def get_ansatz_ws(nqubits: int, parameters: ParameterVector, features: ParameterVector, reps: int=3) -> QuantumCircuit:
    assert len(parameters) == 2 * nqubits
    ansatz = QuantumCircuit(nqubits)
    ansatz.compose(get_ansatz_block(nqubits, parameters), range(nqubits), inplace=True)
    for i in range(1, reps):
        ansatz.barrier()
        ansatz.compose(get_encoding_block(nqubits, features), range(nqubits), inplace=True)
        ansatz.barrier()
        ansatz.compose(get_ansatz_block(nqubits, parameters), range(nqubits), inplace=True)
    return ansatz


In [ ]:
from qiskit.utils import algorithm_globals
algorithm_globals.random_seed = 528491

size = (10, 4)
data = algorithm_globals.random.random(size)
nqubits = data.shape[1]
print(data, nqubits)

In [ ]:
input = ParameterVector("x", data.shape[1])
weights = ParameterVector("theta", nqubits * 2)
circuit_ws = get_ansatz_ws(nqubits, weights, input)
circuit_ws.draw()


In [ ]:
random_weights_ws = algorithm_globals.random.random(2 * nqubits)
print(random_weights_ws)

In [ ]:
from qiskit_machine_learning.neural_networks import SamplerQNN

sqnn_ws = SamplerQNN(
    circuit=circuit_ws,
    input_params=input,
    weight_params=weights,
    interpret=lambda x: "{:b}".format(x).count('1') % 2 == 0, # parity check
    output_shape=2
)
print(sqnn_ws)

In [ ]:
sampler_qnn_forward = sqnn_ws.forward(data[0], random_weights_ws) # require encoding + ansatz parameters, result is a ndarray
print(f"Forward pass result for SamplerQNN: {sampler_qnn_forward}. \nShape: {sampler_qnn_forward.shape}")


In [ ]:
input1 = ParameterVector("x1", data.shape[1])
weights1 = ParameterVector("theta1", nqubits * 2 * 3)
circuit = get_ansatz(nqubits, weights1, input1)
circuit.draw()


In [ ]:
def parity(x):
    print(x, str(x), "{:b}".format(x), "{:b}".format(x).count('1') % 2)
    return "{:b}".format(x).count('1') % 2

def custom_interpret(x):
    return "{:b}".format(x).count('1') % 4


sqnn = SamplerQNN(
    circuit=circuit,
    input_params=input1,
    weight_params=weights1,
    interpret=lambda x: custom_interpret(x), # parity check
    output_shape=4
)
print(sqnn)

In [ ]:
random_weights = algorithm_globals.random.random(len(weights1))
sampler_qnn_forward = sqnn.forward(data[0], random_weights) # require encoding + ansatz parameters, result is a ndarray
print(f"Forward pass result for SamplerQNN: {sampler_qnn_forward}. \nShape: {sampler_qnn_forward.shape}")

In [ ]:
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.quantum_info import SparsePauliOp
from qiskit.opflow.expectations import PauliExpectation

def get_Z_expectation_qubitwise(nqubits: int) -> list:
    """
        :nqubits
        the number of qubits to evaluate
        :return
        the qubitwise observables for each Z expectation value, which follows the formula P(1) = (1 - Exp) / 2
    """
    obs = []
    for i in range(nqubits):
        string = "I" * i + "Z" + "I" * (nqubits - (i + 1))
        obs.append(SparsePauliOp.from_list([(string, 1)]))
    return obs

ob = get_Z_expectation_qubitwise(4)
print(ob)

# observable1 = SparsePauliOp.from_list([("Z", 1), ("Z", 1), ("Z", 1), ("I", 1)])
eqnn_ws = EstimatorQNN(
    circuit=circuit_ws,
    input_params=input,
    weight_params=weights,
    observables=ob
)

print(eqnn_ws)
circuit_ws.draw()

In [ ]:
result = eqnn_ws.forward(data[0], random_weights_ws)
print(result)

# Classic AutoEncoder Wrapper

In [ ]:
import torch.nn as nn
import torch
from sklearn.ensemble import IsolationForest

class HAE(nn.Module):
    """
        general structure is:
        - encoder, FC input_size -> 54 -> 4
        - qnn
        - decoder, FC 4 -> 54 -> input_size
        - tanh activations
    """

    def __init__(self, qnn, input_size: int, nqubits: int = 4) -> None:
        super(HAE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 54),
            nn.Tanh(),
            nn.Linear(54, nqubits),
            nn.Tanh()
        )
        self.vqc = TorchConnector(qnn)
        self.decoder = nn.Sequential(
            nn.Linear(nqubits, 54),
            nn.Tanh(),
            nn.Linear(54, input_size)
        )

        self.isolation_forest = IsolationForest()

    def forward(self, X):
        X = self.encoder(X)
        X = self.vqc(X)
        X = (torch.ones_like(X) - X) / 2
        X = self.decoder(X)
        return X
    
    def encode(self, X):
        C = self.encoder(X)
        return self.vqc(C)
    
    @torch.no_grad()
    def predict(self, X, fit: bool = False):
        code = self.encode(X).cpu()
        if fit:
            self.isolation_forest.fit(code)
        return self.isolation_forest.predict(code) # return +1 if inlier and -1 otherwise

## Using TorchQuantum

In [ ]:
import torchquantum as tq
import torchquantum.functional as tqf

def get_encoder(nqubits):
    pass

def get_ansatz(nqubits):
    pass

def get_block(nqubits, reps):

class HAEBottleneck(nn.Module):
    def __init__(self, nqubits: int = 4, reps):
        self.nqubits = nqubits
        self.quantum_device = tq.QuantumDevice(self.nqubits)

In [ ]:
ins = 4
input = torch.rand((100, ins))
print(input)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
test = HAE(eqnn_ws, input_size=279).to(device)

# Some Training & Evaluation functions

In [ ]:
a = torch.Tensor()
print(a)
a = torch.cat((a, torch.Tensor(2, 1, 1)), dim=0)
torch.cat((a, torch.Tensor(2, 1, 1)), dim=0)

In [ ]:
from tqdm import tqdm
from torch.optim import Adam
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

def evaluate(model, data_loader):
    mse = nn.MSELoss()
    model.eval()
    avg_error = 0
    num_matches = 0
    data = torch.Tensor().to(device)
    labels = []
    with torch.no_grad():
        for X, y in tqdm(data_loader, desc="Validating", leave=False):
            X = X.to(device)
            # reconstruction error
            reconstruction = model(X)
            avg_error += mse(reconstruction, X).sum().item() / len(X)
            data = torch.cat((data, X), dim=0)
            labels += y.tolist()

        predictions = model.predict(data, False)
        for i in range(len(predictions)):
            if predictions[i] == labels[i] == 1:
                num_matches += 1

    return avg_error, num_matches / len(data_loader.dataset)

def training(model, train_dl, val_dl, epochs: int = 100):
    mse = nn.MSELoss()
    optim = Adam(model.parameters(), lr=0.001)
    model.train()
    if_data = torch.Tensor().to(device)
    if_labels = []
    model_loaded = False
    for i in range(1, epochs + 1):
        for X, y in tqdm(train_dl, "Epoch #{}".format(i), leave=True):
            X = X.to(device)
            if not model_loaded:
                if_data = torch.cat((if_data, X), dim=0) 
                if_labels += y.tolist() 
            reconstruction = model(X)
            loss = mse(reconstruction, X)

            optim.zero_grad()
            loss.backward()
            optim.step()

        model_loaded = True
        # if i % 5 == 0:
        # first fit the isolation forest
        train_predictions = model.predict(if_data, True)
        num_matches = 0
        for i in range(len(train_predictions)):
            if train_predictions[i] == if_labels[i] == 1:
                num_matches += 1
        train_accuracy = num_matches / len(train_dl.dataset)
        rec_error, val_accuracy = evaluate(model, val_dl)
        print("Validation average reconstruction error: {}\nTrain anomaly detection accuracy: {}\nValidation anomaly detection accuracy: {}".format(rec_error, train_accuracy, val_accuracy))
        model.train()

# Data
Define a random dataset and the arrhytmia dataset loader

In [ ]:
from torch.utils.data import DataLoader, Dataset, Subset
import numpy as np
class RandomDataset(Dataset):
    def __init__(self, size, length, mean = 0, std_dev = 1) -> None:
        super().__init__()
        self.values = torch.normal(mean, std_dev, size=(length, size))
        self.labels = (torch.normal(0, 1, size=(length,)) > 0) * 1
        self.labels.tolist()

    def __len__(self):
        return len(self.values)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        X = self.values[idx]
        y = self.labels[idx]
        return X, y
    
random_data = RandomDataset(160, 1000, 100, 5)

In [ ]:
import pandas as pd
"""class ArrythmiaDS(Dataset):

    def __init__(self, path: str = "./datasets/arrhythmia.data", get_anomalies: bool = False) -> None:

        def _is_nominal(df, idx) -> bool:
            l = df.iloc[:, idx]
            return len(l) == len(l[l == 0]) + len(l[l == 1])
    
        def _fix_missing(df: pd.DataFrame):
            for i in range(len(df.columns)):
                mean_value = df.iloc[:, i].mean(skipna=True)
                if _is_nominal(df, i):
                    # the mean is the bernoulli probability
                    df.iloc[:, i].map(lambda x: x if x is not pd.NA else 1 * (np.random.random(mean_value) > 0.5))
                else:
                    pass
                    df.iloc[:, i].fillna(value=mean_value, inplace=True)
            return df


        super().__init__()
        self.data = pd.read_csv(path, sep=',', na_values='?', dtype=np.float32)
        self.labels = self.data.iloc[:, -1]
        self.data = self.data.iloc[:, :-1]
        self.data = _fix_missing(self.data)

        # get normal data or anomalies
        if not get_anomalies:
            self.data = self.data[self.labels == 1] 
            self.labels = self.labels[self.labels == 1] 
        else:
            self.data = self.data[self.labels != 1] 
            self.labels = self.labels[self.labels != 1] * 0 # use label = 0 as a generic indicator of anomaly

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data.iloc[idx, :]), torch.tensor(self.labels.iloc[idx])"""
    

"""def get_splits(dataset, dataset_anomalies, test_split: float = 0.3, validation_split: float = 0.3, batch_size: int = 64) -> tuple[DataLoader, DataLoader, DataLoader]:
    I = np.random.permutation(len(dataset))
    Ian = np.random.permutation(len(dataset_anomalies))
    test_size = int(len(dataset) * test_split)
    a_test_size = int(len(dataset_anomalies) * test_split)
    train_val_size = int((len(dataset) - test_size) * validation_split)
    a_train_val_size = int((len(dataset_anomalies) - a_test_size) * validation_split)
    ds_test = Subset(dataset, I[:test_size]) + Subset(dataset_anomalies, Ian[:a_test_size])
    ds_val = Subset(dataset, I[test_size: test_size + train_val_size]) + Subset(dataset_anomalies, Ian[a_test_size: a_test_size + a_train_val_size])
    ds_train = Subset(dataset, I[test_size + train_val_size:])
    return DataLoader(ds_train, batch_size=batch_size, shuffle=True), DataLoader(ds_val, batch_size=batch_size, shuffle=True), DataLoader(ds_test, batch_size=batch_size, shuffle=True)"""

class ArrythmiaDS(Dataset):

    def __init__(self, path: str = "./datasets/arrhythmia.data") -> None:

        def _is_nominal(df, idx) -> bool:
            l = df.iloc[:, idx]
            return len(l) == len(l[l == 0]) + len(l[l == 1])
    
        def _fix_missing(df: pd.DataFrame):
            for i in range(len(df.columns)):
                mean_value = df.iloc[:, i].mean(skipna=True)
                if _is_nominal(df, i):
                    # the mean is the bernoulli probability
                    df.iloc[:, i].map(lambda x: x if x is not pd.NA else 1 * (np.random.random(mean_value) > 0.5))
                else:
                    pass
                    df.iloc[:, i].fillna(value=mean_value, inplace=True)
            return df


        super().__init__()
        self.data = pd.read_csv(path, sep=',', na_values='?', dtype=np.float32)
        self.labels = self.data.iloc[:, -1]
        self.data = self.data.iloc[:, :-1]
        self.data = _fix_missing(self.data)
    
        self.labels[self.labels != 1] = 0 # use label = 0 as a generic indicator of anomaly

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data.iloc[idx, :]), torch.tensor(self.labels.iloc[idx])

def get_splits(dataset, test_split: float = 0.3, validation_split: float = 0.3, batch_size: int = 64) -> tuple[DataLoader, DataLoader, DataLoader]:
    I = np.random.permutation(len(dataset))
    test_size = int(len(dataset) * test_split)
    train_val_size = int((len(dataset) - test_size) * validation_split)
    ds_test = Subset(dataset, I[:test_size])
    ds_val = Subset(dataset, I[test_size: test_size + train_val_size])
    ds_train = Subset(dataset, I[test_size + train_val_size:])
    return DataLoader(ds_train, batch_size=batch_size, shuffle=True), DataLoader(ds_val, batch_size=batch_size, shuffle=True), DataLoader(ds_test, batch_size=batch_size, shuffle=True)

ds = ArrythmiaDS()
train_dl, val_dl, test_dl = get_splits(ds, batch_size=4)

# A quick test
We will only use two qubits as latent space

In [ ]:
train_features, train_labels = next(iter(train_dl))
print(train_features.shape, train_labels)

In [ ]:
from qiskit.primitives import Estimator
reps = 3
nqubits = 2
weights = ParameterVector("w", 2 * nqubits * reps) # 2n * reps
inputs = ParameterVector("x", nqubits) # n
qnn_circuit = get_ansatz(nqubits, parameters=weights, features=inputs, reps=reps)
qnn = EstimatorQNN(
    circuit=qnn_circuit,
    input_params=inputs,
    weight_params=weights,
    input_gradients=True,
    observables=get_Z_expectation_qubitwise(nqubits),
    # estimator=Estimator(options={'shots': 20})
)

In [ ]:
hae = HAE(qnn, 279, nqubits=nqubits).to(device)

In [ ]:
training(hae, train_dl, val_dl, epochs=100)

In [ ]:
import torchquantum as tq
import torchquantum.functional as tqf

def get_encoder(nqubits):
    pass

def get_ansatz(nqubits):
    pass

def get_block(nqubits, reps):

class HAEBottleneck(nn.Module):
    def __init__(self, nqubits: int = 4, reps):
        self.nqubits = nqubits
        self.quantum_device = tq.QuantumDevice(self.nqubits)

In [ ]:
import torchquantum as tq
import torchquantum.functional as tqf
import torch.nn as nn

class HAEBottleneck(nn.Module):
    # initially non parametrized, circuit 10 with 3 reps
    def __init__(self):
        super(HAEBottleneck, self).__init__()
        self.qd = tq.QuantumDevice(n_wires=4) # 4 is the number of qubits
        self.encoder = [tqf.rx] * 4
        self.measure = tq.MeasureAll(tq.PauliZ)

        # BLOCK 1
        self.ansatz_1_y_1 = tq.ry(has_param=True, trainable=True)
        self.ansatz_1_x_1 = tq.rx(has_params=True, trainable=True)

        self.ansatz_1_y_2 = tq.ry(has_params=True, trainable=True)
        self.ansatz_1_x_2 = tq.rx(has_params=True, trainable=True)
        
        self.ansatz_1_y_3 = tq.ry(has_params=True, trainable=True)
        self.ansatz_1_x_3 = tq.rx(has_params=True, trainable=True)
        
        self.ansatz_1_y_4 = tq.ry(has_params=True, trainable=True)
        self.ansatz_1_x_4 = tq.rx(has_params=True, trainable=True)

        # BLOCK 2
        self.ansatz_2_y_1 = tq.ry(has_params=True, trainable=True)
        self.ansatz_2_x_1 = tq.rx(has_params=True, trainable=True)

        self.ansatz_2_y_2 = tq.ry(has_params=True, trainable=True)
        self.ansatz_2_x_2 = tq.rx(has_params=True, trainable=True)
        
        self.ansatz_2_y_3 = tq.ry(has_params=True, trainable=True)
        self.ansatz_2_x_3 = tq.rx(has_params=True, trainable=True)
        
        self.ansatz_2_y_4 = tq.ry(has_params=True, trainable=True)
        self.ansatz_2_x_4 = tq.rx(has_params=True, trainable=True)

        # BLOCK 3
        self.ansatz_3_y_1 = tq.ry(has_params=True, trainable=True)
        self.ansatz_3_x_1 = tq.rx(has_params=True, trainable=True)

        self.ansatz_3_y_2 = tq.ry(has_params=True, trainable=True)
        self.ansatz_3_x_2 = tq.rx(has_params=True, trainable=True)
        
        self.ansatz_3_y_3 = tq.ry(has_params=True, trainable=True)
        self.ansatz_3_x_3 = tq.rx(has_params=True, trainable=True)
        
        self.ansatz_3_y_4 = tq.ry(has_params=True, trainable=True)
        self.ansatz_3_x_4 = tq.rx(has_params=True, trainable=True)

    def forward(self, X):
        batch_size = X.shape[0]

        # BLOCK 1
        self.ansatz_1_y_1(self.qd, wires=0)
        self.ansatz_1_x_1(self.qd, wires=0)

        self.ansatz_1_y_2(self.qd, wires=1)
        self.ansatz_1_x_2(self.qd, wires=1)
        
        self.ansatz_1_y_3(self.qd, wires=2)
        self.ansatz_1_x_3(self.qd, wires=2)
        
        self.ansatz_1_y_4(self.qd, wires=3)
        self.ansatz_1_x_4(self.qd, wires=3)

        self.qd.cnot(wires=[3, 0])
        self.qd.cnot(wires=[0, 1])
        self.qd.cnot(wires=[1, 2])
        self.qd.cnot(wires=[2, 3])

        # ENCODER
        for i, gate in enumerate(self.encoder):
            gate(self.qd, wires=i, params=X[:, i])

        # BLOCK 2
        self.ansatz_2_y_1(self.qd, wires=0)
        self.ansatz_2_x_1(self.qd, wires=0)

        self.ansatz_2_y_2(self.qd, wires=1)
        self.ansatz_2_x_2(self.qd, wires=1)
        
        self.ansatz_2_y_3(self.qd, wires=2)
        self.ansatz_2_x_3(self.qd, wires=2)
        
        self.ansatz_2_y_4(self.qd, wires=3)
        self.ansatz_2_x_4(self.qd, wires=3)

        self.qd.cnot(wires=[3, 0])
        self.qd.cnot(wires=[0, 1])
        self.qd.cnot(wires=[1, 2])
        self.qd.cnot(wires=[2, 3])

        # ENCODER
        for i, gate in enumerate(self.encoder):
            gate(self.qd, wires=i, params=X[:, i])

        # BLOCK 1
        self.ansatz_3_y_1(self.qd, wires=0)
        self.ansatz_3_x_1(self.qd, wires=0)

        self.ansatz_3_y_2(self.qd, wires=1)
        self.ansatz_3_x_2(self.qd, wires=1)
        
        self.ansatz_3_y_3(self.qd, wires=2)
        self.ansatz_3_x_3(self.qd, wires=2)
        
        self.ansatz_3_y_4(self.qd, wires=3)
        self.ansatz_3_x_4(self.qd, wires=3)

        self.qd.cnot(wires=[3, 0])
        self.qd.cnot(wires=[0, 1])
        self.qd.cnot(wires=[1, 2])
        self.qd.cnot(wires=[2, 3])

        X = self.measure(self.qd).reshape(batch_size, 4)
        return X

In [ ]:
import torch 
input = torch.rand((10, 4))
m = HAEBottleneck()
output = m(input)
print(input, output)

In [ ]:
torch.save(hae.state_dict(), "weights/hae_last.pt")

In [ ]:
w = torch.load("weights/hae_3reps_160.pt")
print(type(w))

In [ ]:
hae.load_state_dict(w)

In [ ]:
evaluate(hae, test_dl)

In [ ]:
for i, (x, y) in enumerate(test_dl):
    print(x.shape)

In [ ]:
import torch
import torchquantum as tq
import torchquantum.functional as tqf
import torch.nn as nn
from sklearn.ensemble import IsolationForest
from torch.utils.data import DataLoader, Dataset, Subset
import numpy as np

device = 'cpu'

encoder_list = [
    {'input_idx': [0], 'func': 'rx', 'wires': [0]},
    {'input_idx': [1], 'func': 'rx', 'wires': [1]},
    {'input_idx': [2], 'func': 'rx', 'wires': [2]},
    {'input_idx': [3], 'func': 'rx', 'wires': [3]}
]

from tqdm import tqdm
from torch.optim import Adam

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)


def evaluate(model, data_loader):
    mse = nn.MSELoss()
    model.eval()
    avg_error = 0
    num_matches = 0
    data = torch.Tensor().to(device)
    labels = []
    with torch.no_grad():
        for X, y in tqdm(data_loader, desc="Validating", leave=False):
            X = X.to(device)
            # reconstruction error
            reconstruction = model(X)
            avg_error += mse(reconstruction, X).sum().item() / len(X)
            data = torch.cat((data, X), dim=0)
            labels += y.tolist()

        predictions = model.predict(data, False)
        for i in range(len(predictions)):
            if predictions[i] == labels[i]:
                num_matches += 1

    return avg_error, num_matches / len(data_loader.dataset)


def training(model, train_dl, val_dl, epochs: int = 100):
    mse = nn.MSELoss()
    optim = Adam(model.parameters(), lr=0.00001)
    model.train()
    if_data = torch.Tensor().to(device)
    if_labels = []
    model_loaded = False
    for i in range(1, epochs + 1):
        for X, y in tqdm(train_dl, "Epoch #{}".format(i), leave=True):
            X = X.to(device)
            if not model_loaded:
                if_data = torch.cat((if_data, X), dim=0)
                if_labels += y.tolist()
            reconstruction = model(X)
            loss = mse(reconstruction, X)

            optim.zero_grad()
            loss.backward()
            optim.step()

        model_loaded = True
        # if i % 5 == 0:
        # first fit the isolation forest
        train_predictions = model.predict(if_data, True)
        num_matches = 0
        for i in range(len(train_predictions)):
            if train_predictions[i] == if_labels[i] == 1:
                num_matches += 1
        train_accuracy = num_matches / len(train_dl.dataset)
        rec_error, val_accuracy = evaluate(model, val_dl)
        print(
            "\nValidation average reconstruction error: {}\nTrain anomaly detection accuracy: {}\nValidation anomaly detection accuracy: {}".format(
                rec_error, train_accuracy, val_accuracy))
        model.train()


class HAEBottleneck(tq.QuantumModule):
    # initially non parametrized, circuit 10 with 3 reps
    def __init__(self):
        super(HAEBottleneck, self).__init__()
        self.qd = tq.QuantumDevice(n_wires=4)  # 4 is the number of qubits
        self.encoder = tq.GeneralEncoder(encoder_list)
        self.measure = tq.MeasureAll(obs=tq.PauliZ)
        self.observables = [
            [tq.PauliZ(), tq.I(), tq.I(), tq.I()],
            [tq.I(), tq.PauliZ(), tq.I(), tq.I()],
            [tq.I(), tq.I(), tq.PauliZ(), tq.I()],
            [tq.I(), tq.I(), tq.I(), tq.PauliZ()]
        ]

        # BLOCK 1
        self.ansatz_1_y_1 = tq.RY(has_params=True, trainable=True)
        self.ansatz_1_x_1 = tq.RX(has_params=True, trainable=True)

        self.ansatz_1_y_2 = tq.RY(has_params=True, trainable=True)
        self.ansatz_1_x_2 = tq.RX(has_params=True, trainable=True)

        self.ansatz_1_y_3 = tq.RY(has_params=True, trainable=True)
        self.ansatz_1_x_3 = tq.RX(has_params=True, trainable=True)

        self.ansatz_1_y_4 = tq.RY(has_params=True, trainable=True)
        self.ansatz_1_x_4 = tq.RX(has_params=True, trainable=True)

        # BLOCK 2
        self.ansatz_2_y_1 = tq.RY(has_params=True, trainable=True)
        self.ansatz_2_x_1 = tq.RX(has_params=True, trainable=True)

        self.ansatz_2_y_2 = tq.RY(has_params=True, trainable=True)
        self.ansatz_2_x_2 = tq.RX(has_params=True, trainable=True)

        self.ansatz_2_y_3 = tq.RY(has_params=True, trainable=True)
        self.ansatz_2_x_3 = tq.RX(has_params=True, trainable=True)

        self.ansatz_2_y_4 = tq.RY(has_params=True, trainable=True)
        self.ansatz_2_x_4 = tq.RX(has_params=True, trainable=True)

        # BLOCK 3
        self.ansatz_3_y_1 = tq.RY(has_params=True, trainable=True)
        self.ansatz_3_x_1 = tq.RX(has_params=True, trainable=True)

        self.ansatz_3_y_2 = tq.RY(has_params=True, trainable=True)
        self.ansatz_3_x_2 = tq.RX(has_params=True, trainable=True)

        self.ansatz_3_y_3 = tq.RY(has_params=True, trainable=True)
        self.ansatz_3_x_3 = tq.RX(has_params=True, trainable=True)

        self.ansatz_3_y_4 = tq.RY(has_params=True, trainable=True)
        self.ansatz_3_x_4 = tq.RX(has_params=True, trainable=True)

        # BLOCK 4
        self.ansatz_4_y_1 = tq.RY(has_params=True, trainable=True)
        self.ansatz_4_x_1 = tq.RX(has_params=True, trainable=True)

        self.ansatz_4_y_2 = tq.RY(has_params=True, trainable=True)
        self.ansatz_4_x_2 = tq.RX(has_params=True, trainable=True)

        self.ansatz_4_y_3 = tq.RY(has_params=True, trainable=True)
        self.ansatz_4_x_3 = tq.RX(has_params=True, trainable=True)

        self.ansatz_4_y_4 = tq.RY(has_params=True, trainable=True)
        self.ansatz_4_x_4 = tq.RX(has_params=True, trainable=True)

        # BLOCK 4
        self.ansatz_5_y_1 = tq.RY(has_params=True, trainable=True)
        self.ansatz_5_x_1 = tq.RX(has_params=True, trainable=True)

        self.ansatz_5_y_2 = tq.RY(has_params=True, trainable=True)
        self.ansatz_5_x_2 = tq.RX(has_params=True, trainable=True)

        self.ansatz_5_y_3 = tq.RY(has_params=True, trainable=True)
        self.ansatz_5_x_3 = tq.RX(has_params=True, trainable=True)

        self.ansatz_5_y_4 = tq.RY(has_params=True, trainable=True)
        self.ansatz_5_x_4 = tq.RX(has_params=True, trainable=True)

    def forward(self, X: torch.Tensor):
        def cnot_block():
            tqf.cnot(self.qd, wires=[3, 0])
            tqf.cnot(self.qd, wires=[0, 1])
            tqf.cnot(self.qd, wires=[1, 2])
            tqf.cnot(self.qd, wires=[2, 3])

        batch_size = X.shape[0]
        # BLOCK 1
        self.ansatz_1_y_1(self.qd, wires=0)
        self.ansatz_1_x_1(self.qd, wires=0)

        self.ansatz_1_y_2(self.qd, wires=1)
        self.ansatz_1_x_2(self.qd, wires=1)

        self.ansatz_1_y_3(self.qd, wires=2)
        self.ansatz_1_x_3(self.qd, wires=2)

        self.ansatz_1_y_4(self.qd, wires=3)
        self.ansatz_1_x_4(self.qd, wires=3)

        cnot_block()

        # ENCODER
        self.encoder(self.qd, X)

        # BLOCK 2
        self.ansatz_2_y_1(self.qd, wires=0)
        self.ansatz_2_x_1(self.qd, wires=0)

        self.ansatz_2_y_2(self.qd, wires=1)
        self.ansatz_2_x_2(self.qd, wires=1)

        self.ansatz_2_y_3(self.qd, wires=2)
        self.ansatz_2_x_3(self.qd, wires=2)

        self.ansatz_2_y_4(self.qd, wires=3)
        self.ansatz_2_x_4(self.qd, wires=3)

        cnot_block()

        # ENCODER
        self.encoder(self.qd, X)

        # BLOCK 1
        self.ansatz_3_y_1(self.qd, wires=0)
        self.ansatz_3_x_1(self.qd, wires=0)

        self.ansatz_3_y_2(self.qd, wires=1)
        self.ansatz_3_x_2(self.qd, wires=1)

        self.ansatz_3_y_3(self.qd, wires=2)
        self.ansatz_3_x_3(self.qd, wires=2)

        self.ansatz_3_y_4(self.qd, wires=3)
        self.ansatz_3_x_4(self.qd, wires=3)
        # ENCODER
        self.encoder(self.qd, X)

        # BLOCK 1
        self.ansatz_4_y_1(self.qd, wires=0)
        self.ansatz_4_x_1(self.qd, wires=0)

        self.ansatz_4_y_2(self.qd, wires=1)
        self.ansatz_4_x_2(self.qd, wires=1)

        self.ansatz_4_y_3(self.qd, wires=2)
        self.ansatz_4_x_3(self.qd, wires=2)

        self.ansatz_4_y_4(self.qd, wires=3)
        self.ansatz_4_x_4(self.qd, wires=3)
        # ENCODER
        self.encoder(self.qd, X)

        # BLOCK 1
        self.ansatz_5_y_1(self.qd, wires=0)
        self.ansatz_5_x_1(self.qd, wires=0)

        self.ansatz_5_y_2(self.qd, wires=1)
        self.ansatz_5_x_2(self.qd, wires=1)

        self.ansatz_5_y_3(self.qd, wires=2)
        self.ansatz_5_x_3(self.qd, wires=2)

        self.ansatz_5_y_4(self.qd, wires=3)
        self.ansatz_5_x_4(self.qd, wires=3)

        """
        e0 = tq.expval(self.qd, [i for i in range(4)], self.observables[0]) # 1 x B
        e1 = tq.expval(self.qd, [i for i in range(4)], self.observables[1])
        e2 = tq.expval(self.qd, [i for i in range(4)], self.observables[2])
        e3 = tq.expval(self.qd, [i for i in range(4)], self.observables[3])
        E = torch.stack((e0[:, 0], e1[:, 1], e2[:, 2], e3[:, 3]), dim=0).T  # 4 x B
        print((e0[:, 0], e1[:, 1], e2[:, 2], e3[:, 3]))"""
        Z = self.measure(self.qd)
        """print(Z, E-Z)

        X = torch.stack((e0[:, 0], e1[:, 1], e2[:, 2], e3[:, 3]), dim=0) # 4 x B
        print(X.shape, X)"""
        return Z


class HAE(nn.Module):
    """
        general structure is:
        - encoder, FC input_size -> 54 -> 4
        - qnn
        - decoder, FC 4 -> 54 -> input_size
        - tanh activations
    """

    def __init__(self, input_size: int, nqubits: int = 4) -> None:
        super(HAE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_size, 54),
            nn.Tanh(),
            nn.Linear(54, nqubits),
            nn.Tanh()
        )
        self.vqc = HAEBottleneck()
        self.decoder = nn.Sequential(
            nn.Linear(nqubits, 54),
            nn.Tanh(),
            nn.Linear(54, input_size)
        )

        self.isolation_forest = IsolationForest()

    def forward(self, X):
        X = self.encoder(X)
        X = self.vqc(X)
        # X = (torch.ones_like(X) - X) / 2
        X = self.decoder(X)
        return X

    def encode(self, X):
        C = self.encoder(X)
        return self.vqc(C)

    @torch.no_grad()
    def predict(self, X, fit: bool = False):
        code = self.encode(X).cpu()
        if fit:
            self.isolation_forest.fit(code)
        return self.isolation_forest.predict(code)  # return +1 if inlier and -1 otherwise


import pandas as pd

"""class ArrythmiaDS(Dataset):

    def __init__(self, path: str = "./datasets/arrhythmia.data", get_anomalies: bool = False) -> None:

        def _is_nominal(df, idx) -> bool:
            l = df.iloc[:, idx]
            return len(l) == len(l[l == 0]) + len(l[l == 1])

        def _fix_missing(df: pd.DataFrame):
            for i in range(len(df.columns)):
                mean_value = df.iloc[:, i].mean(skipna=True)
                if _is_nominal(df, i):
                    # the mean is the bernoulli probability
                    df.iloc[:, i].map(lambda x: x if x is not pd.NA else 1 * (np.random.random(mean_value) > 0.5))
                else:
                    pass
                    df.iloc[:, i].fillna(value=mean_value, inplace=True)
            return df


        super().__init__()
        self.data = pd.read_csv(path, sep=',', na_values='?', dtype=np.float32)
        self.labels = self.data.iloc[:, -1]
        self.data = self.data.iloc[:, :-1]
        self.data = _fix_missing(self.data)

        # get normal data or anomalies
        if not get_anomalies:
            self.data = self.data[self.labels == 1] 
            self.labels = self.labels[self.labels == 1] 
        else:
            self.data = self.data[self.labels != 1] 
            self.labels = self.labels[self.labels != 1] * 0 # use label = 0 as a generic indicator of anomaly

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data.iloc[idx, :]), torch.tensor(self.labels.iloc[idx])"""

"""def get_splits(dataset, dataset_anomalies, test_split: float = 0.3, validation_split: float = 0.3, batch_size: int = 64) -> tuple[DataLoader, DataLoader, DataLoader]:
    I = np.random.permutation(len(dataset))
    Ian = np.random.permutation(len(dataset_anomalies))
    test_size = int(len(dataset) * test_split)
    a_test_size = int(len(dataset_anomalies) * test_split)
    train_val_size = int((len(dataset) - test_size) * validation_split)
    a_train_val_size = int((len(dataset_anomalies) - a_test_size) * validation_split)
    ds_test = Subset(dataset, I[:test_size]) + Subset(dataset_anomalies, Ian[:a_test_size])
    ds_val = Subset(dataset, I[test_size: test_size + train_val_size]) + Subset(dataset_anomalies, Ian[a_test_size: a_test_size + a_train_val_size])
    ds_train = Subset(dataset, I[test_size + train_val_size:])
    return DataLoader(ds_train, batch_size=batch_size, shuffle=True), DataLoader(ds_val, batch_size=batch_size, shuffle=True), DataLoader(ds_test, batch_size=batch_size, shuffle=True)"""


class ArrythmiaDS(Dataset):

    def __init__(self, path: str = "datasets/arrhythmia.data", normalize: bool = True) -> None:

        def _is_nominal(df, idx) -> bool:
            l = df.iloc[:, idx]
            return len(l) == len(l[l == 0]) + len(l[l == 1])

        def _fix_missing(df: pd.DataFrame):
            for i in range(len(df.columns)):
                mean_value = df.iloc[:, i].mean(skipna=True)
                if _is_nominal(df, i):
                    # the mean is the bernoulli probability
                    df.iloc[:, i].map(lambda x: x if x is not pd.NA else 1 * (np.random.random(mean_value) > 0.5))
                else:
                    pass
                    df.iloc[:, i].fillna(value=mean_value, inplace=True)
            return df

        super().__init__()
        self.data = pd.read_csv(path, sep=',', na_values='?', dtype=np.float32)
        self.labels = self.data.iloc[:, -1]
        self.data = self.data.iloc[:, :-1]
        self.data = _fix_missing(self.data)
        # print("------------", self.data.max() - self.data.min(), "\n---------------",self.data.std())
        # self.data = (self.data - self.data.min()) / (self.data.max() - self.data.min()) # standard normalization
        self.data = self.data / (self.data.abs().max() + 1e-6)
        self.labels[self.labels != 1] = -1  # use label = -1 as a generic indicator of anomaly

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data.iloc[idx, :]), torch.tensor(self.labels.iloc[idx])


class CensusDS(Dataset):

    def __init__(self, path: str = "datasets/census-income-binarized.csv") -> None:
        super().__init__()
        self.data = pd.read_csv(path, sep=',', na_values='?', dtype=np.float32, skiprows=0)
        self.labels = self.data.iloc[:, -1]
        self.data = self.data.iloc[:, :-1]
        self.labels[self.labels == 1] = -1  # use label = -1 as a generic indicator of anomaly
        self.labels[self.labels == 0] = 1  # use label = -1 as a generic indicator of anomaly


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data.iloc[idx, :]), torch.tensor(self.labels.iloc[idx])


def get_splits(dataset, test_split: float = 0.1, validation_split: float = 0.3, batch_size: int = 4) -> tuple[
    DataLoader, DataLoader, DataLoader]:
    I = np.random.permutation(len(dataset))
    test_size = int(len(dataset) * test_split)
    train_val_size = int((len(dataset) - test_size) * validation_split)
    ds_test = Subset(dataset, I[:test_size])
    ds_val = Subset(dataset, I[test_size: test_size + train_val_size])
    ds_train = Subset(dataset, I[test_size + train_val_size:])
    return DataLoader(ds_train, batch_size=batch_size, shuffle=True), DataLoader(ds_val, batch_size=batch_size,
                                                                                 shuffle=True), DataLoader(ds_test,
                                                                                                           batch_size=batch_size,
                                                                                                           shuffle=True)


"""
ds = ArrythmiaDS()
train_dl, val_dl, test_dl = get_splits(ds, batch_size=6)

input = torch.rand((100, 40))
m = HAE(input_size=279).to('cuda')
training(m, train_dl, val_dl, epochs=200)
torch.save(m.state_dict(), "param.pt")
"""
ds = CensusDS()
train_dl, val_dl, test_dl = get_splits(ds, batch_size=80)
m = HAE(input_size=ds[0][0].shape[0]).to('cuda')
training(m, train_dl, val_dl, epochs=100)
torch.save(m.state_dict(), "param_census.pt")
